In [1]:
debug_show_dataset_in_gui = False

# 📦 Intital imports
import sys
from pathlib import Path

# Zoek project root waar config.py staat
project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent
sys.path.insert(0, project_root.as_posix())

# Voeg ook src/ toe aan sys.path
SRC_DIR = project_root / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Importeer paden en hulpfuncties
from config import SRC_DIR, CSV_DIR, SHAPEFILES_DIR, EXPORTS_DIR, UTILS_DIR
from notebook_setup import init_notebook_environment
from ui_helpers import setup_tab_controls

# Init-actie (optioneel print info)
init_notebook_environment()

ℹ️ SRC al aanwezig of niet gevonden: /Users/hvingen/Repositories/OU_Project/covid_dashboard_6/src


In [2]:
# 📦 Widget imports
import ipywidgets as widgets

In [3]:
# 📂 Import clean COVID dataset
import data_service
df = data_service.get_prepared_covid_dataset()

# print("Beschikbare kolommen in df:", df.columns.tolist())

In [4]:
# 📂 Paden instellen + modules laden
import os, sys
from pathlib import Path

project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent
sys.path.insert(0, project_root.as_posix())

from config import SRC_DIR, CSV_DIR
if SRC_DIR.as_posix() not in sys.path:
    sys.path.insert(0, SRC_DIR.as_posix())

import data_service 
from data_service import get_metric_mapping_covid, get_metric_mapping_sewer, get_available_years
import covid_dashboard_presenter
from IPython.display import display
from ipywidgets import Button, Layout, Textarea, HBox, VBox

from data_service import get_province_heatmap_data
from data_service import get_municipality_heatmap_data
from data_service import get_province_heatmap_riool_data
from data_service import get_municipality_heatmap_riool_data

from covid_dashboard_presenter import plot_province_heatmap
from covid_dashboard_presenter import plot_municipality_heatmap
from covid_dashboard_presenter import plot_province_heatmap_riool
from covid_dashboard_presenter import plot_municipality_heatmap_riool

In [5]:
# Let Municipalities checkbox flip exclusive
def on_municipalities_change(change):
    if change['new'] == True:
        months_checkbox.value = False

# Let Months checkbox flip exclusive
def on_months_change(change):
    if change['new'] == True:
        municipalities_checkbox.value = False

# Plot Covid GUI
def plot_covid(year, total_reported, hospital_admission, deceased, province, municipalities, months):
    covid_dashboard_presenter.plot_covid(df,
                                        year, 
                                        total_reported, 
                                        hospital_admission, 
                                        deceased, 
                                        province, 
                                        municipalities, 
                                        months)  

def update_province_heatmap(year, column):
    gdf = get_province_heatmap_data(year)
    plot_province_heatmap(gdf, column=column)

# Functie om de heatmap bij te werken
def update_municipality_heatmap(year, column):
    from utils.debug_utils import print_debug_summary, print_merge_status, print_column_names, print_debug_table
    from data_service import get_municipality_heatmap_data, get_prepared_covid_dataset

    gdf = get_municipality_heatmap_data(year, column)
    plot_municipality_heatmap(gdf, column=column)

    if debug_show_dataset_in_gui:
        df = get_prepared_covid_dataset()
        df_grouped = df[df['Year'] == year].groupby('Municipality_merged')[[column]].sum().reset_index()

        print_debug_summary(gdf, column)
        print_merge_status(gdf, df_grouped, key='Municipality_merged')
        print_column_names(gdf)
        print_debug_table(df_grouped, 'Municipality_merged', column, title=f"Covid-data per gemeente in {year}")


# Plot Rioolwater GUI
def plot_covid_riool(year):
    covid_dashboard_presenter.plot_covid_riool(year)  

def update_province_heatmap(year, column):
    from utils.debug_utils import print_debug_summary, print_merge_status, print_column_names, print_debug_table
    from data_service import get_province_heatmap_data, get_prepared_covid_dataset
    gdf = get_province_heatmap_data(year, column)
    plot_province_heatmap(gdf, column=column)
    if debug_show_dataset_in_gui:
        df = get_prepared_covid_dataset()
        df_grouped = df[df['Year'] == year].groupby('Province_merged')[[column]].sum().reset_index()
        print_debug_summary(gdf, column)
        print_merge_status(gdf, df_grouped, key='Province_merged')
        print_column_names(gdf)
        print_debug_table(df_grouped, 'Province_merged', column, title=f"Covid-data per provincie in {year}")    

In [6]:
# Create a list of outs for tabs
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

# Create a list of tabs for GUI
tabs = widgets.Tab(children = [out1, out2, out3, out4])
tabs.set_title(0, 'Covid Barcharts')
tabs.set_title(1, 'Covid Heatmap')
tabs.set_title(2, 'Sewer Heatmap')
tabs.set_title(3, 'Download dataset')

# print('Werkdirectory ingesteld op:', os.getcwd())

In [7]:
# 📊 Tab 1, Plot Covid barcharts
with out1:
    title_widget1 = widgets.HTML("<h3 style='text-align:left; margin-left:90px;'>COVID-Dataset</h3>")
    year_dropdown = widgets.Dropdown(description='Year:', options=get_available_years(df))
    total_reported_checkbox = widgets.Checkbox(value=True, description='Total reported')
    hospital_admission_checkbox = widgets.Checkbox(value=True, description='Hospital admission')
    deceased_checkbox = widgets.Checkbox(value=True, description='Deceased')
    province_dropdown = widgets.Dropdown(description='Level:', options=['Netherlands', 'All provinces'] + df['Province_merged'].dropna().unique().tolist(), value='Netherlands')
    municipalities_checkbox = widgets.Checkbox(value=True, description='Municipalities')
    months_checkbox = widgets.Checkbox(value=False, description='Months')
    months_checkbox.observe(on_months_change, names='value')
    municipalities_checkbox.observe(on_municipalities_change, names='value')

    row1 = VBox( [year_dropdown, total_reported_checkbox, hospital_admission_checkbox, deceased_checkbox, province_dropdown, municipalities_checkbox, months_checkbox])
    ui = VBox( [title_widget1, row1])
    out = widgets.interactive_output(plot_covid, {
        'year':year_dropdown, 
        'total_reported':total_reported_checkbox, 
        'hospital_admission':hospital_admission_checkbox, 
        'deceased':deceased_checkbox, 
        'province': province_dropdown, 
        'municipalities': municipalities_checkbox, 
        'months': months_checkbox
    } )
    display(ui, out)

In [8]:
# 📊 Tab 2, Plot Covid heatmap
with out2:              
    title_widget2 = widgets.HTML("<h3 style='text-align:left; margin-left:90px;'>Heatmap Provinces</h3>")       
    year_dropdown = widgets.Dropdown(description='Year:', options=get_available_years(df))    
    region_dropdown = widgets.Dropdown(description='Level:', options=['Provinces', 'Municipalities'], value='Provinces')       
    metric_options = get_metric_mapping_covid()
    metric_dropdown = widgets.Dropdown(description='Metric:', options=list(metric_options.keys()), value='Total reported')
    
    def update_heatmap(year, metric_label, region_type):
        from data_service import get_prepared_covid_dataset
        from utils.debug_utils import print_debug_summary, print_merge_status, print_column_names, print_debug_table
    
        title_text = 'Covid Heatmap ' + region_type
        title_widget2.value = f"<h3 style='text-align:left; margin-left:90px;'>{title_text}</h3>"
        metric_options = get_metric_mapping_covid()
        column = metric_options[metric_label]
    
        if region_type == 'Provinces':
            gdf = get_province_heatmap_data(year, column)
            plot_province_heatmap(gdf, column=column)
    
            if debug_show_dataset_in_gui:
                df = get_prepared_covid_dataset()
                df_grouped = df[df['Year'] == year].groupby('Province_merged')[[column]].sum().reset_index()
                print_debug_summary(gdf, column)
                print_merge_status(gdf, df_grouped, key='Province_merged')
                print_column_names(gdf)
                print_debug_table(df_grouped, 'Province_merged', column, title=f"Covid-data per provincie in {year}")
    
        else:
            gdf = get_municipality_heatmap_data(year, column)
            plot_municipality_heatmap(gdf, column=column)
    
            if debug_show_dataset_in_gui:
                df = get_prepared_covid_dataset()
                df_grouped = df[df['Year'] == year].groupby('Municipality_name_merged')[[column]].sum().reset_index()
                print_debug_summary(gdf, column)
                print_merge_status(gdf, df_grouped, key='Municipality_name_merged')
                print_column_names(gdf)
                print_debug_table(df_grouped, 'Municipality_name_merged', column, title=f"Covid-data per gemeente in {year}")
            
    ui = VBox([title_widget2, year_dropdown, region_dropdown, metric_dropdown])
    out = widgets.interactive_output(update_heatmap, {
        'year': year_dropdown,
        'metric_label': metric_dropdown,
        'region_type': region_dropdown
    })
    display(ui, out)

In [9]:
# 📊 Tab 3, Plot Sewer heatmap
with out3:
    title_widget3 = widgets.HTML("<h3 style='text-align:left; margin-left:80px;'>Heatmap Riool</h3>")
    year_dropdown = widgets.Dropdown(description='Year:', options=get_available_years(df))
    region_dropdown = widgets.Dropdown(description='Level:', options=['Provinces', 'Municipalities'], value='Provinces')
    metric_options = get_metric_mapping_sewer()
    metric_dropdown = widgets.Dropdown(description='Metric:', options=list(metric_options.keys()), value='RNA flow per 100000')

    def update_heatmap_riool(year, metric_label, region_type):
        from data_service import get_prepared_riool_dataset
        from utils.debug_utils import print_debug_summary, print_merge_status, print_column_names, print_debug_table
    
        title_text = 'Sewer Heatmap ' + region_type
        title_widget3.value = f"<h3 style='text-align:left; margin-left:90px;'>{title_text}</h3>"
        metric_options = get_metric_mapping_sewer()
        column = metric_options[metric_label]
    
        if region_type == 'Provinces':
            gdf = get_province_heatmap_riool_data(year, column)
            plot_province_heatmap_riool(gdf, column=column)
    
            if debug_show_dataset_in_gui:
                df = get_prepared_riool_dataset().aantallen_riool
                df = df.rename(columns={'RWZI_AWZI_name': 'Gemeentenaam'})
                df_prov = get_prepared_riool_dataset().gemeenten_per_provincie
                df_merged = df.merge(df_prov, on='Gemeentenaam', how='inner')
                df_grouped = df_merged[df_merged['Year'] == year].groupby('Provincie')[[column]].sum().reset_index()
                gdf = gdf.rename(columns={'PROV_NAAM': 'Provincie'})
                print_debug_summary(gdf, column)
                print_merge_status(gdf, df_grouped, key='Provincie')
                print_column_names(gdf)
                print_debug_table(df_grouped, 'Provincie', column, title=f"Riooldata per provincie in {year}")
    
        else:
            gdf = get_municipality_heatmap_riool_data(year, column)
            plot_municipality_heatmap_riool(gdf, column=column)

            if debug_show_dataset_in_gui:
                df = get_prepared_riool_dataset().aantallen_riool
                df = df.rename(columns={'RWZI_AWZI_name': 'Gemeentenaam'})
                df_grouped = df[df['Year'] == year].groupby('Gemeentenaam')[[column]].sum().reset_index()
                gdf = gdf.rename(columns={'GM_NAAM': 'Gemeentenaam'})
                print_debug_summary(gdf, column)
                print_merge_status(gdf, df_grouped, key='RNA_flow_per_100000')
                print_column_names(gdf)
                print_debug_table(df_grouped, 'Gemeentenaam', column, title=f"Riooldata per gemeente in {year}")

    ui = VBox([title_widget3, year_dropdown, region_dropdown, metric_dropdown])
    out = widgets.interactive_output(update_heatmap_riool, {
        'year': year_dropdown,
        'region_type': region_dropdown,
        'metric_label': metric_dropdown
    })
    display(ui, out)

In [10]:
# 📦 Tab 4, Download datasets
with out4:
    # Tab 4 - Download RIVM-data
    title_widget4 = widgets.HTML("<h3 style='text-align:left; margin-left:90px;'>Download RIVM-data</h3>")
    from IPython.display import display, Markdown
    from data_writer import Saveframes
    def run_download(*args):
        output.clear_output()
        
        try:
            result = Saveframes()
            for fname, status in result.items():
                print(f"{fname}: {status}")
        except Exception as e:
            print(f"❌ Fout bij opslaan of ophalen van resultaten: {e}")

    download_button = widgets.Button(description='Download datasets', button_style='success')
    download_button.on_click(run_download)

    output = widgets.Output()
    display(widgets.VBox([
        title_widget4,
        download_button,
        output
    ]))

In [11]:
# ✅ Display Tabs Covid Dashboard
display(tabs)